# Image Processing Template: On-Chip Binding
___
Duncan Muir

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from htbam_analysis.processing import chip
from htbam_analysis.processing import experiment as exp
from htbam_analysis.processing import chipcollections as collections
from pathlib import Path 

%config InlineBackend.figure_format = 'svg'

## 1. Establish experiment and pinlist

In [ ]:
root = '/Volumes/DuncanSSD/20231212_new_binding_code/'
description = 'adpr_binding_mac1'
operator = 'DFM'
setup_num = 's1'
device_num = 'd1'
device_dimensions = (32, 56)
e = exp.Experiment(description, root, operator)

### Set Exposure and Channel Params

In [ ]:
button_quant_channel = '2'
button_quant_exposure = 5

## Mock a Pinlist for Flow-On Experiments

In [ ]:
pinlist = e.read_pinlist("/Users/duncanmuir/Downloads/20230426_mac1_p1_p2_print_pinlist.csv")
pinlist.head(5)

## 2. Add devices and corners

In [ ]:
# Top left, top right, bottom left, bottom right
d1_corners = ((396, 398),(6665,373),(424,6778),(6689,6749)) 

#Do this in imageJ, put your cursor over the reaction chambers and copy over
d1 = exp.Device(setup_num, device_num, device_dimensions, pinlist, d1_corners)


### Non-standard chamber-finding parameters

In [ ]:
#chip.Stamp.circlePara1Index = 30
#chip.Stamp.circlePara2Index = 15

print('Old Chamber Radius: {} pixels'.format(chip.Stamp.chamberrad))
chip.Stamp.chamberrad = 32#33 in 2x2
print('New Chamber Radius: {} pixels'.format(chip.Stamp.chamberrad))

## 3. Execute button analyses

#### Quantify egfp fluorescence on buttons

##### Load

In [ ]:
# Stitched image path
button_quant_path = "/Volumes/DuncanSSD/20231212_new_binding_code/button_quant/20231205-162503-d1_button_quant_2x2_sensitivity_4x/2/StitchedImages/BGSubtracted_StitchedImg_5_2_0.tif"

# Prepare ChipQuant object
d1_GFPQuant = collections.ChipQuant(d1, 'ButtonReference')

# Load image into memory
d1_GFPQuant.load_file(button_quant_path, button_quant_channel, button_quant_exposure)

##### Process

In [ ]:
# Find buttons, extract intensity attributes
d1_GFPQuant.process()

# Summarize attributes as a pandas dataframe
quant_report1 = d1_GFPQuant.summarize()

##### Save Summary/Image

In [ ]:
quant_report1.to_csv(f'{root}/data_analysis/button_quant_summary.csv', index=True)

d1_GFPQuant.save_summary_image()

In [ ]:
#code is having a hard time finding chambers in the standard curve (perhaps because it was resized?), so using this image as a reference

d1_ChamberRef = collections.ChipQuant(d1, 'Chamber_Ref')
d1_ChamberRef.load_file(button_quant_path, '2', 5)

In [ ]:
d1_ChamberRef.process(mapped_features = 'chamber', coerce_center = True)


### 7. Binding Assay

Note: This is only applicable if you ran a binding experiment (obviously).

There are three images taken for each concentration of prey: 
1. A pre-wash measurement of the bait.
2. A post-wash measurement of the bait.
3. A post-wash measurement of the prey.

I process these images by first re-formatting the directory outputteed by the binding experiment into a format identical to that of a kinetics experiment. I've included some code for doing this in the cell below. Keep in mind this may require some tweaks depending on how your files or directories are named. For instance, the `get_handle()` and `concen_from_handle()` functions used to extract experiment data from filenames will almost certainly need to be tailored for your specific experiment.

This is a hacky solution that will probably work alright until a newer version of the processing code is published. 

#### Re-format the Binding Experiment Directory

In [ ]:
# create necessary directories, copy files, and rename files programatically

import os 
import shutil 
import numpy as np

binding_path = '/Volumes/DuncanSSD/20231212_new_binding_code/adpr_binding/'
bait_channel, bait_exposure = '2', '5'
prey_channel, prey_exposure = '4', '1'

pre_wash_bait_dir = os.path.join(binding_path, 'pre_wash_bait')
post_wash_bait_dir = os.path.join(binding_path, 'post_wash_bait')
post_wash_prey_dir = os.path.join(binding_path, 'post_wash_prey')

def get_handle(filename: str):

    substring = filename.split('-')[-1]
    substring = substring.split('_P')[0]
    substring = substring.split('d1')[-1]
    
    return substring[1:]

def concen_from_handle(handle: str, prey_name: str):

    concen_string = handle.split(prey_name)[0][0:-1]
    concen_string = concen_string.replace('_', '.')

    return float(concen_string)

source_dirs = os.listdir(binding_path)
source_dirs.sort()
#source_dirs = source_dirs[0:-1]
file_handles = [get_handle(dir) for dir in source_dirs]
concens = [concen_from_handle(handle, 'ADPR') for handle in file_handles]

target_dirs = np.array([''] * len(file_handles), dtype='<U80')
target_dirs[0::3] = pre_wash_bait_dir
target_dirs[1::3] = post_wash_bait_dir
target_dirs[2::3] = post_wash_prey_dir

exposures = np.array([''] * len(file_handles), dtype='<U3')
exposures[0::3] = bait_exposure
exposures[1::3] = bait_exposure
exposures[2::3] = prey_exposure

channels = np.array([''] * len(file_handles), dtype='<U3')
channels[0::3] = bait_channel
channels[1::3] = bait_channel
channels[2::3] = prey_channel

def add_nested_folders(target_dirs: np.ndarray, channels: np.ndarray):
    
    nested_folders = []
    for dir, channel in zip(target_dirs, channels):

        channel_subdir = os.path.join(dir, channel)
        images_subdir = os.path.join(channel_subdir, 'StitchedImages')
        nested_folders.append(images_subdir)

        if not os.path.exists(dir):
            os.mkdir(dir)
            os.mkdir(channel_subdir)
            os.mkdir(images_subdir)

        else:
            print(f'ERROR: {dir} already exists!')

    return nested_folders
target_dirs = add_nested_folders(target_dirs, channels)
for fh in file_handles:
    print(fh)
for index in range(len(file_handles)):

    source_dir = source_dirs[index]
    handle = file_handles[index]
    concen = concens.index(concens[index])
    target_dir = target_dirs[index]
    exposure = exposures[index]
    channel = channels[index]

    target_filename = 'BGSubtracted_StitchedImg_{}_{}_{}.tif'
    target_filename = target_filename.format(
        exposure,
        channel,
        concen
    )
    target_filename = os.path.join(binding_path, target_dir, target_filename)

    source_filename = os.path.join(
        binding_path, 
        source_dir, 
        channel, 
        'StitchedImages',
        'StitchedImg_{}_{}_{}.tif'.format(exposure, channel, 0)
        )

    shutil.copyfile(source_filename, target_filename)

#### Establish Button Reference

In [ ]:
# button_ref_image = 'path/to/button/ref/image.tif'
# button_ref_channel, button_ref_exposure = '2', 10

# button_ref = collections.ChipQuant(d3, 'button_ref')
# button_ref.load_file(button_ref_image, button_ref_channel, button_ref_exposure)
# button_ref.process(mapped_features = 'button')
# button_ref.save_summary_image()

#### Process Bait and Prey Images

In [ ]:
# process each image type as if it were a collection of kinetic images
binding_path = '/Volumes/DuncanSSD/20231212_new_binding_code/adpr_binding/'
bait_channel, bait_exposure = '2', '5'
prey_channel, prey_exposure = '4', '1'

# first, process the bait
bait_handles = ['pre_wash_bait', 'post_wash_bait']
binding_bait = collections.ButtonChamberAssaySeries(
    device=d1,
    descriptions=bait_handles,
    chamber_ref=d1_ChamberRef.chip,
    button_ref=d1_GFPQuant.chip,
    channels=[bait_channel]
)
binding_bait.parse_kineticsFolders(
    root=binding_path,
    file_handles=bait_handles,
    descriptors=bait_handles,
    channel=bait_channel,
    exposure=bait_exposure,
    pattern="{}*/{}/StitchedImages")
binding_bait.process_kinetics(featuretype='button', low_mem=False)
binding_bait.save_summary()

# then, process the prey 
prey_handles = ['post_wash_prey']
binding_prey = collections.ButtonChamberAssaySeries(
    device=d1,
    descriptions=prey_handles,
    chamber_ref=d1_ChamberRef.chip,
    button_ref=d1_GFPQuant.chip,
    channels=[prey_channel]
)
binding_prey.parse_kineticsFolders(
    root=binding_path,
    file_handles=prey_handles,
    descriptors=prey_handles,
    channel=prey_channel,
    exposure=prey_exposure,
    pattern="{}*/{}/StitchedImages"
)
binding_prey.process_kinetics(featuretype='button', low_mem=False)
binding_prey.save_summary()